In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import statistics
from pathlib import Path
import datetime

import requests
import re
import os
import sys


from multiprocess import Pool
import psutil
from tqdm import tqdm

from bs4 import BeautifulSoup

import glob

import matplotlib.pyplot as plt

In [29]:
import networkx as nx


import pydot
from networkx.drawing.nx_pydot import graphviz_layout

from community import community_louvain as community

In [ ]:
#data = pd.read_csv('your_path_to_file/sample_date.csv.gzip', compression = 'gzip')

In [157]:
a_nodes = data.query("message_hash in fwd_message_hash and not (message_hash != message_hash)") 

In [139]:
#defining sample size, as full dataset takes longer than 12 hours
a_nodes = a_nodes.head(2000)

In [140]:
a_tup = tuple([a_nodes['group_name'].tolist(), a_nodes['message_hash'].tolist(), a_nodes['author'].tolist()])

In [141]:
a_edge = []
for i in range(len(a_tup[0])):
    try:
        tmp = data['group_name'][(data['fwd_message_hash'] == a_tup[1][i]) & (data['fwd_author'] == a_tup[2][i])]
        a = tmp.tolist()[0]
        
    except:
        continue
    
    a_edge.append((a_tup[0][i], a))

In [142]:
G = nx.complete_graph(1)

G.add_nodes_from(a_tup[0])

G.add_edges_from(a_edge)

nodes_to_remove = [node for node in G.nodes() if G.degree(node) < 1]

G.remove_nodes_from(nodes_to_remove)

In [145]:
partition = community.best_partition(G)

In [ ]:
pos = nx.spring_layout(G)
fig, ax = plt.subplots(figsize=(24,20))

nx.draw(G, pos, ax = ax, with_labels=False, node_size=30, cmap=plt.cm.jet, node_color=list(partition.values()))
plt.show()

In [ ]:
# Compute the best partition
partition = community.best_partition(G)

# Draw the graph
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=30,
                       cmap=plt.cm.jet, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.7)



In [149]:
#close_centrality = nx.closeness_centrality(G)
#bet_centrality = nx.betweenness_centrality(G, normalized = True,
#											endpoints = False)

In [151]:
#mean degree
degrees = [val for (node, val) in G.degree()]
sum(degrees)/len(degrees)

5.4146868250539955

In [161]:
#modularity
mod = community.modularity(partition, G)

In [155]:
#centrality
deg_centrality = nx.degree_centrality(G)

cent = []
for i in deg_centrality:
    cent.append(deg_centrality[i])